## Parte 1: Modelagem Conceitual 

### 1.1. Diagrama ER

In [ ]:
from IPython.display import Image, display

# URL da imagem
url = "https://i.postimg.cc/nz0mBWkS/der-tp1-BDA-drawio.png"

# Exibir a imagem
display(Image(url=url))

### 1.2. Decisões de modelagem
- Relacionameto  **CLIENTE - PEDIDO: 1:N**\
Este relacionamento é 1:N, pois um cliente pode realizar diversos pedidos, entretanto, um pedido só pode estar vinculado a um único cliente.\
Além disso, um cliente pode nao realizar nenhum pedido, mas todo pedido deve estar associado a um cliente.

- Relacionamento **PEDIDO - PRODUTO: N:M**\
Este relacionamento é N:M, pois um pedido pode conter diversos produtos e um tipo de produto pode estar em diversos pedidos diferentes.\
Além disso, todo pedido deve conter produtos, mas nem todo produto precisa estar em um pedido.

- Relacionamento **PRODUTO - CARGA: N:M**\
Foi-se considerado como carga o conjunto de itens comprados. Exemplo: comprou-se os itens a, b e c, logo a carga será o pacote com os itens do pedido.  
Este relacionamento é N:M, pois um mesmo tipo de produto pode estar em diversas cargas diferentes, e uma mesma carga pode conter vários tipos de produtos diferentes.
Além disso, toda carga deve conter produtos, mas nem todo produto precisa estar em uma carga.

- Relacionamento **CLIENTE - ENDEREÇO: N:1**\
Este relacionamento é N:1, pois um cliente , mas em um mesmo endereço pode residir diversos clientes.
Além disso, todo cliente deve residir em um endereço, mas nem todo endereço necessita ter um cliente.

- Relacionamento **CARGA - VEÍCULO: N:1**\
Este relacionamento é N:1, pois uma carga é transportada por apenas um veículo, mas um veículo pode transportar várias cargas.
Além disso, toda carga deve ser transportada por um veículo, mas nem todo veículo precisa transportar uma carga.

- Relacionamento **VEÍCULO - MOTORISTA: 1:1**:\
Este relacionamento é 1:1, pois um veículo está associado a apenas um motorista e um motorista está associado a apenas o seu único veículo.
Além disso, um veículo pode não estar associado a nenhum motorista, e podem haver motoristas não associados a nenhum veículo.

- Relacionamento **MOTORISTA - ROTAS: 1:N**:\
Este relacionamento é 1:N, pois um mesmo motorista pode realizar múltiplas rotas diferentes, mas uma rota está associada a um único motorista.
Além disso, este é um relacionamento fraco, uma vez que se motorista for removido do banco de dados, a rota não irá existir pois não haverá ninguém para realizá-la.

- Relacionamto **ENDEREÇO - CARGA: 1:N (Tem origem):**\
Este relacionamento é 1:N, pois um mesmo endereço pode ser origem de várias cargas diferentes, mas uma carga específica só possui um endereço de origem.
Além disso, toda carga necessariamente tem origem em um endereço, mas nem todo endereço precisa ser a origem de uma carga.

- Relacionamto **ENDEREÇO - CARGA: 1:N (Tem destino):**\
Este relacionamento é 1:N, pois um mesmo endereço pode ser destino de várias cargas diferentes, mas uma carga específica só possui um endereço de destino.
Além disso, toda carga necessariamente tem um endereço como destino, mas nem todo endereço precisa ser o destino de uma carga.



## Parte 2: Esquema Relacional

In [ ]:
from IPython.display import Image, display

# URL da imagem
url = "https://i.postimg.cc/9MnMWH12/relacional-tp1-BDA-drawio.png"

# Exibir a imagem
display(Image(url=url))

## Parte 3: Álgebra Relacional

## Parte 4: SQL

### 4.1. Criação do Banco

In [ ]:
import sqlite3

conn = sqlite3.connect("meubanco.db")
cursor = conn.cursor()

# cria a tabela
cursor.executescript("""
DROP TABLE IF EXISTS Endereco;
DROP TABLE IF EXISTS Cliente;
DROP TABLE IF EXISTS Produto;
DROP TABLE IF EXISTS Pedido;
DROP TABLE IF EXISTS Veiculo;
DROP TABLE IF EXISTS Motorista;
DROP TABLE IF EXISTS Rotas;
DROP TABLE IF EXISTS Carga;
DROP TABLE IF EXISTS Pedido_Produto;
DROP TABLE IF EXISTS Carga_Produto;
               
CREATE TABLE Endereco (
    id_endereco   INTEGER PRIMARY KEY,
    nome_rua      TEXT,
    num_rua       INTEGER,
    complemento   TEXT,
    cep           INTEGER,
    bairro        TEXT
);

CREATE TABLE Cliente (
    cpf_cliente   INTEGER PRIMARY KEY,
    nome_cliente  TEXT,
    id_endereco   INTEGER,
    FOREIGN KEY (id_endereco) REFERENCES Endereco(id_endereco)
);
               
CREATE TABLE Produto (
    id_produto    INTEGER PRIMARY KEY,
    nome_produto  TEXT,
    preco_produto    INTEGER
);
               
CREATE TABLE Pedido (
    id_pedido     INTEGER PRIMARY KEY,
    data_pedido   TEXT,
    total_pedido  INTEGER,
    entrega_prevista TEXT,
    cpf_cliente INTEGER,
    FOREIGN KEY (cpf_cliente) REFERENCES Cliente(cpf_cliente)
);
               
CREATE TABLE Veiculo (
    id_veiculo    INTEGER PRIMARY KEY,
    nome_veiculo  TEXT,
    tipo_veiculo  TEXT
);

CREATE TABLE Motorista (
    cpf_motorista INTEGER PRIMARY KEY,
    nome_motorista TEXT,
    id_veiculo    INTEGER,
    FOREIGN KEY (id_veiculo) REFERENCES Veiculo(id_veiculo)
);

CREATE TABLE Rotas (
    cpf_motorista TEXT,
    id_rota INTEGER,
    nome_rota TEXT,
    PRIMARY KEY (cpf_motorista, id_rota),   -- chave composta
    FOREIGN KEY (cpf_motorista) 
        REFERENCES Motorista(cpf_motorista) 
        ON DELETE CASCADE
);
               
CREATE TABLE Carga (
    id_carga            INTEGER PRIMARY KEY,
    id_endereco_origem  INTEGER,
    id_endereco_destino INTEGER,
    id_veiculo          INTEGER,
    status_carga        TEXT,
    data_entrega        TEXT,
    FOREIGN KEY (id_endereco_origem)  REFERENCES Endereco(id_endereco),
    FOREIGN KEY (id_endereco_destino) REFERENCES Endereco(id_endereco),
    FOREIGN KEY (id_veiculo)          REFERENCES Veiculo(id_veiculo)
);
               
CREATE TABLE Pedido_Produto (
    id_pedido     INTEGER,
    id_produto    INTEGER,
    qtd_pedida    INTEGER,
    total_produto INTEGER,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido)  REFERENCES Pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES Produto(id_produto)
);               

CREATE TABLE Carga_Produto (
    id_carga   INTEGER,
    id_produto INTEGER,
    tam_carga  INTEGER,
    PRIMARY KEY (id_carga, id_produto),
    FOREIGN KEY (id_carga)   REFERENCES Carga(id_carga),
    FOREIGN KEY (id_produto) REFERENCES Produto(id_produto)
);
""")

conn.commit()


### 4.2. Inserção de Dados

In [ ]:
cursor.executescript("""                                    
-- ENDERECO
INSERT INTO Endereco (id_endereco, nome_rua, num_rua, complemento, cep, bairro) VALUES
(1, 'Rua das Acácias', 101, 'Apto 201', 30130000, 'Funcionários'),
(2, 'Av. Carandaí', 1115, '13º andar', 30130915, 'Funcionários'),
(3, 'Rua da Bahia', 900, NULL, 30160011, 'Centro'),
(4, 'Av. Amazonas', 3500, 'Sala 402', 30420000, 'Barro Preto'),
(5, 'Rua Padre Eustáquio', 1200, NULL, 30720000, 'Padre Eustáquio'),
(6, 'Av. Cristiano Machado', 12345, 'Bloco B', 31720000, 'Cidade Nova');

-- CLIENTE
INSERT INTO Cliente (cpf_cliente, nome_cliente, id_endereco) VALUES
(1001, 'Alice Souza', 1),
(1002, 'Bruno Lima', 2),
(1003, 'Carla Mendes', 3),
(1004, 'Diego Castro', 4),
(1005, 'Eva Pereira', 5);

-- PRODUTO
INSERT INTO Produto (id_produto, nome_produto, preco_produto) VALUES
(1, 'Notebook 14"', 3500),
(2, 'Mouse Óptico', 60),
(3, 'Teclado Mecânico', 420),
(4, 'Monitor 24"', 900),
(5, 'Headset USB', 150),
(6, 'Cabo HDMI', 40);

-- VEICULO
INSERT INTO Veiculo (id_veiculo, nome_veiculo, tipo_veiculo) VALUES
(1, 'Van 01', 'Van'),
(2, 'Caminhão 3/4', 'Caminhao'),
(3, 'Fiorino 07', 'Furgão'),
(4, 'HR 2.5', 'Caminhonete'),
(5, 'Sprinter 415', 'Van');

-- MOTORISTA
INSERT INTO Motorista (cpf_motorista, nome_motorista, id_veiculo) VALUES
(2001, 'João Batista', 1),
(2002, 'Marcos Paulo', 2),
(2003, 'Nina Rocha', 3),
(2004, 'Otávio Reis', 4),
(2005, 'Paula Dias', 5);

-- ROTAS - entidade fraca, depende do motorista
INSERT INTO Rotas (cpf_motorista, id_rota, nome_rota) VALUES
(2001, 1, 'Centro–Funcionários'),
(2001, 2, 'Funcionários–Savassi'),
(2002, 1, 'Barreiro–Centro'),
(2003, 1, 'Pampulha–Centro'),
(2004, 1, 'Cidade Nova–Centro'),
(2005, 1, 'Padre Eustáquio–Barro Preto');

-- PEDIDO
INSERT INTO Pedido (id_pedido, data_pedido, total_pedido, entrega_prevista, cpf_cliente) VALUES
(5001, '2025-08-15', 4460, '2025-09-01', 1001),
(5002, '2025-08-16',  200, '2025-09-02', 1002),
(5003, '2025-08-17',  940, '2025-09-03', 1003),
(5004, '2025-08-18', 3650, '2025-09-04', 1004),
(5005, '2025-08-19',  190, '2025-09-05', 1005);

-- CARGA
INSERT INTO Carga (id_carga, id_endereco_origem, id_endereco_destino, id_veiculo, status_carga, data_entrega) VALUES
(6001, 2, 1, 1, 'entregue',   '2025-08-20'),
(6002, 4, 3, 2, 'em_rota',    NULL),
(6003, 6, 5, 3, 'pendente',   NULL),
(6004, 1, 4, 4, 'entregue',   '2025-08-22'),
(6005, 3, 2, 5, 'em_rota',    NULL);

-- PEDIDO_PRODUTO – chaves compostas (id_pedido, id_produto)
INSERT INTO Pedido_Produto (id_pedido, id_produto, qtd_pedida, total_produto) VALUES
(5001, 1, 1, 3500),   -- Notebook
(5001, 3, 1,  420),   -- Teclado
(5001, 2, 9,   540),  -- Mouses para fechar 4460
(5002, 5, 1,  150),   -- Headset
(5002, 6, 1,   40),   -- HDMI
(5003, 4, 1,  900),   -- Monitor
(5003, 6, 1,   40),
(5004, 1, 1, 3500),
(5004, 2, 2,  120),
(5005, 2, 1,   60),
(5005, 5, 1,  130);

-- CARGA_PRODUTO – vincula cargas aos produtos
INSERT INTO Carga_Produto (id_carga, id_produto, tam_carga) VALUES
(6001, 1, 1),  -- Notebook entregue
(6001, 3, 1),  -- Teclado entregue
(6002, 4, 2),  -- Em rota
(6002, 6, 4),
(6003, 5, 3),  -- Pendente
(6004, 2, 5),  -- Entregue
(6005, 2, 3),  -- Em rota
(6005, 6, 2);
""")

In [ ]:
for t in tabelas:
    print(f"\n=== {t} (até 5 linhas) ===")
    for row in cursor.execute(f"SELECT * FROM {t} LIMIT 5"):
        print(row)

### 4.3. Consultas

In [ ]:
import pandas as pd

#### 4.3.1. Listar todos os clientes com cargas em rota de entrega

In [ ]:
query = """
SELECT DISTINCT
  c.cpf_cliente,
  c.nome_cliente
FROM Cliente c
JOIN Carga cg
  ON cg.id_endereco_destino = c.id_endereco
WHERE cg.status_carga = 'em_rota'
ORDER BY c.nome_cliente;
"""

df = pd.read_sql_query(query, conn)
df

#### 4.3.2. Recuperar o nome dos produtos que nunca foram entregues

In [ ]:
query = """
SELECT p.nome_produto
FROM Produto p
EXCEPT
SELECT p.nome_produto
FROM Produto p
JOIN Carga_Produto cp ON p.id_produto = cp.id_produto
JOIN Carga c          ON cp.id_carga  = c.id_carga
WHERE c.status_carga = 'entregue';  -- ou: c.data_entrega IS NOT NULL
"""

df = pd.read_sql_query(query, conn)
df

#### 4.3.3.  Obter a descrição das cargas transportadas por um mesmo motorista

In [ ]:
query = """
SELECT
  m.cpf_motorista,
  m.nome_motorista,
  c.id_carga,
  c.status_carga,
  c.data_entrega,
  c.id_endereco_origem  AS id_end_origem,
  c.id_endereco_destino AS id_end_destino
FROM Motorista m
JOIN Veiculo  v ON v.id_veiculo = m.id_veiculo
JOIN Carga    c ON c.id_veiculo = v.id_veiculo
ORDER BY m.nome_motorista, c.id_carga;
"""

df = pd.read_sql_query(query, conn)
df

In [ ]:
conn.close()